Reading in all libraries

In [1]:
library("huge")

Loading required package: Matrix
Loading required package: lattice
Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

Loading required package: MASS


Reading in Modified Data File

In [ ]:
expressionData=read.csv("hg-annoHeaderCorrection.csv", header=TRUE, stringsAsFactors = FALSE)

Getting Dimensions of expressionData matrix before cleaning everything up

In [ ]:
print(dim(expressionData))

Getting Duplicated Gene Entries

In [ ]:
duplicatedGenesLogicalIndices=duplicated(expressionData$geneName)
duplicatedGeneNames= expressionData$geneName[duplicatedGenesLogicalIndices]
print("Duplicated Genes:")
print(duplicatedGeneNames)

Removing the duplicated genes from the analysis. This should not affect the analysis very much.

In [ ]:
expressionData=expressionData[!duplicatedGenesLogicalIndices,]
print(dim(expressionData))

In [ ]:
#Getting Expression Values for all Genes across all single cells.
expressionValues=expressionData[,(2:ncol(expressionData))]

#Writing Genes for Later Use by the Java Program and possibly GO Term Analysis
write.table(expressionData$geneName,"geneNamesHuman.csv",quote=FALSE, row.names=FALSE, col.names=FALSE)

#Removing NA values. Hopefully this will resolve the problems with the graph. Right now I am seeing values only 
#along the diagnal. This should hopefully fix that problem.
expressionValues[is.na(expressionValues)]=0

print(dim(expressionValues))

In [ ]:
expressionMatrix=as.matrix(t(expressionValues))
print(dim(expressionMatrix))

In [ ]:
allGeneNames=sapply(expressionData$geneName,toupper)
colnames(expressionMatrix)=allGeneNames
commonMouseAndHumanGenes=unlist(read.csv("commonMouseAndHumanGenes.txt",header=FALSE))
expressionMatrix=expressionMatrix[,commonMouseAndHumanGenes]
print(dim(expressionMatrix))

In [ ]:
#Reducing the set of genes even more to see what happens. This code is behaving in a very weird way. What am I doing wrong?
randomGenes=sample(1:ncol(expressionMatrix),size=1000,replace=FALSE)
expressionMatrix=expressionMatrix[,randomGenes]

Randomly sampling 500 cells twice from expressionMatrix. The objective is to see if I get similar or idential graphs when I do this and take the intersection. This is kind of a sanity check to see if the result of the intersection with the human dataset makes sense.

In [ ]:
randomCells1=sample(1:nrow(expressionMatrix),size = 1000,replace=FALSE)
print(head(randomCells1))
randomCells2=sample(1:nrow(expressionMatrix),size = 1000,replace=FALSE)
print(head(randomCells2))
randomCellsFromExpressionMatrix1=expressionMatrix[randomCells1,]
print(dim(randomCellsFromExpressionMatrix1))
randomCellsFromExpressionMatrix2=expressionMatrix[randomCells2,]
print(dim(randomCellsFromExpressionMatrix2))

In [ ]:
print("Everything except Graph model done")

graphModel1 = huge(randomCellsFromExpressionMatrix1, method="glasso", lambda=c(0.055))
output1=as.matrix(graphModel1$icov[[1]])
colnames(output1)=colnames(expressionMatrix)
write.table(output1,"graph_outputForRandomSample1.csv", sep=",", quote=FALSE, row.names = FALSE)
print("Done First!")

graphModel2 = huge(randomCellsFromExpressionMatrix2, method="glasso", lambda=c(0.055))
output2=as.matrix(graphModel2$icov[[1]])
colnames(output2)=colnames(expressionMatrix)
write.table(output2,"graph_outputForRandomSample2.csv", sep=",", quote=FALSE, row.names = FALSE)

print("Done Second!")
#######################################################################################################

In [ ]:
print(dim(graphModel1$icov[[1]]))
print(dim(graphModel2$icov[[1]]))

Getting Some Statistics on the Generated Graphs:

In [ ]:
#Getting Adjacency matrices and setting the diagnal to FALSE i.e. no self edges.
logicalOutput1=(output1!=0)
diag(logicalOutput1)=FALSE
logicalOutput2=(output2!=0)
diag(logicalOutput2)=FALSE
pdf("Edge_Distribution_Statistics.pdf")
edgeDistribution1=apply(logicalOutput1,2,sum)
hist(edgeDistribution1,100, main = "Edge Distribution for First Random Sample")
edgeDistribution2=apply(logicalOutput2,2,sum)
hist(edgeDistribution2,100, main = "Edge Distribution for Second Random Sample")

#Getting Intersection of both outputs
intersectofOutputs=((logicalOutput1+logicalOutput2)>1)
diag(intersectofOutputs)=FALSE

intersectEdgeDistribution=apply(intersectofOutputs,2,sum)
hist(intersectEdgeDistribution,100, main = "Edge Distribution for Intersect of Random Samples")
dev.off()

Generating Sparsity Measurements for Each Adjacency Matrix generated in the previous cell.

In [ ]:
sink("sparsityValues.txt")
print("Sparsity for Random Sample 1:")
sparsity1=sum(logicalOutput1)/(nrow(logicalOutput1)*ncol(logicalOutput1))
print(sparsity1)
print("Sparsity for Random Sample 2:")
sparsity2=sum(logicalOutput2)/(nrow(logicalOutput2)*ncol(logicalOutput2))
print(sparsity2)
print("Sparsity for Intersect of Random Samples:")
intersectSparsity=sum(intersectofOutputs)/(nrow(intersectofOutputs)*ncol(intersectofOutputs))
print(intersectSparsity)